В этой программе я анализирую отзывы на Почту России. При 30-ти отзывах каждого типа уникальных слов было мало, так что я брала по 50 отзывов для тренировки и по 10 для проверки качества.  

Плохие отзывы:  
https://www.otzyvru.com/pochta-rossii?rating=1  
Хорошие отзывы:  
https://www.otzyvru.com/pochta-rossii?rating=5

In [1]:
import requests
import re
import time
from nltk import word_tokenize
from bs4 import BeautifulSoup
from pprint import pprint
from sklearn.metrics import accuracy_score
from collections import Counter
from pymorphy2 import MorphAnalyzer
session = requests.session()
morph = MorphAnalyzer()

In [2]:
# собирает список ссылок с одной страницы
# возвращает список ссылок на полные отзывы
def parse_page(link, number):
    responce = session.get(link)
    html = responce.text
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for h in soup.select('h2'):
        if len(links) < number:
            l = h.find('a').attrs['href']
            links.append(l)
    return links

In [3]:
# собирает список ссылок с обеих страниц
# возвращает список ссылок на полные отзывы
def parse_two_pages(link):
    first_p = parse_page(link, 30)
    second_p = parse_page(link+'&page=2', 20)
    full_links = first_p + second_p   
    return full_links

In [4]:
# токенизация и начальная форма
# возвращает список слов
def process(text):
    lem_text = []
    for word in word_tokenize(text.lower()):
        if word.isalpha():
            result = morph.parse(word)[0]
            lemma = result.normal_form
            lem_text.append(lemma)
    return lem_text

In [5]:
# собирает текст отзывов
# возвращает список всех слов отзывов этого типа
def grab_reviews(links):
    revs = []
    for link in links:
        responce = session.get(link)
        html = responce.text
        soup = BeautifulSoup(html, 'html.parser')
        review = soup.find('span', {'class': 'comment description'}).text
        lem_rev = process(review)
        try:
            revs += lem_rev
        except:
            print(revs)
    return revs

In [6]:
# собираем ссылки на все полные отзывы
good = 'https://www.otzyvru.com/pochta-rossii?rating=5'
bad = 'https://www.otzyvru.com/pochta-rossii?rating=1'
good_links = parse_two_pages(good)
bad_links = parse_two_pages(bad)

In [7]:
# собираем списки всех хороших и плохих слов
good_list = grab_reviews(good_links)
bad_list = grab_reviews(bad_links)

In [8]:
# выкидываем пересекающиеся
good_only = [g for g in good_list if g not in bad_list]
bad_only = [b for b in bad_list if b not in good_list]

In [9]:
# собирает фреклист для одного списка слов
def get_freqlist(word_list):
    freqlist = Counter()
    for word in word_list:
        freqlist[word] += 1
    return dict(freqlist.most_common(100))

In [10]:
# собираем хороший и плохой фреклист
freqlists = {}
good_freq = get_freqlist(good_only)
bad_freq = get_freqlist(bad_only)
freqlists['Это положительный отзыв'] = good_freq
freqlists['Это отрицательный отзыв'] = bad_freq

In [11]:
# анализирует текст, выдаёт общий результат и красивенький
def analyze(freqlists, text):
    counts = Counter()
    for sentiment, freqlist in freqlists.items():
        freqlist = Counter(freqlist)
        for word in text:
            counts[sentiment] += int(freqlist[word] > 0)
    result = counts.most_common()
    nice_output = 'Вердикт: '+str(result[0][0])+' с шансами '+str(result[0][1])+' к '+str(result[1][1])
    return result, nice_output

In [12]:
# тестируем
test_text = 'Не прислали извещения, молча отправили обе посылки обратно в Китай'
print(analyze(freqlists, process(test_text))[1])

Вердикт: Это отрицательный отзыв с шансами 3 к 0


In [16]:
# собирает 20 тестовых текстов
# отличается от grab_reviews() тем, что делает подсписки, а не всё в кучу
def grab_test(links):
    revs = []
    for link in links:
        responce = session.get(link)
        html = responce.text
        soup = BeautifulSoup(html, 'html.parser')
        review = soup.find('span', {'class': 'comment description'}).text
        lem_rev = process(review)
        revs.append(lem_rev)
    return revs

In [22]:
# анализирует качество
def test_detection(freqlists):
    test_list = []
    gold = ['Это положительный отзыв']*10+['Это отрицательный отзыв']*10
    results = []
    test_links = ['https://www.otzyvru.com/pochta-rossii?rating=5&page=3','https://www.otzyvru.com/pochta-rossii?rating=1&page=3']
    for link in test_links:
        test_list += grab_test(parse_page(link, 10))
    for text in test_list:
        predicted_sentiment = analyze(freqlists, text)[0][0][0]
        results.append(predicted_sentiment)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [23]:
test_detection(freqlists)

Accuracy: 0.9000


**Идеи улучшения программы:**  
1. В списке уникальных слов из хороших отзывов почти все слова имеют положительную окраску, а в таком же списке из плохих отзывов оказывается очень много нейтральных слов (см. ниже). Я и так увеличила сет с 30 до 50 отзывов каждого типа, но, кажется, хорошо бы сделать его ещё побольше.  
2. Мне не нравится, что функция grab_test() почти дублирует grab_reviews() за исключением другой структуры списка: [[раз, два], [три, четыре]] вместо [раз, два, три, четыре].
3. После подсчёта accuracy ради интереса можно вывести те отзывы, где результат анализа не совпал с правильным.

In [32]:
print('Примеры хороших слов: \n', ', '.join(good_only[:15]))
print('\nПримеры плохих слов: \n', ', '.join(bad_only[:15]))

Примеры хороших слов: 
 приятный, нереальный, условие, ваш, труд, опс, стачка, грамотно, очевидно, отметить, вежливый, улыбаться, благодарить, компетентность, клиентоориентированность

Примеры плохих слов: 
 август, сделать, заказ, алиэкспресс, сей, пора, ни, прийти, отслеживание, прислать, заказ, доходить, ужасный, емs, ижевск
